In [1]:
import shutil
import sys
import os.path
import pandas as pd

# check if pyomo has been installed. If not, install with pip
if not shutil.which("pyomo"):
    !pip install -q pyomo
assert(shutil.which("pyomo"))

# check if GLPK is installed. If not, install.
if not (shutil.which("glpsol") or os.path.isfile("glpsol")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk
        except:
            pass
assert(shutil.which("glpsol") or os.path.isfile("glpsol"))

# Selección de la ubicación de almacenes
La ubicación de almacenes es un problema típico de optimización discreta. Una empresa está considerando una serie de ubicaciones para construir almacenes que abastezcan a un conjunto de tiendas. Cada almacén tiene un coste de explotación fijo y una capacidad máxima que especifica el número de tiendas que puede atender. Además, cada tienda debe ser abastecida exactamente por un almacén y el coste de abastecer una tienda depende del almacén seleccionado. 

El modelo consiste en elegir qué almacenes construir y qué almacén asignar a cada tienda para minimizar el coste total, es decir, la suma de los costes fijos y de suministro. 

## Datos
Este problema se va a modelar como un `AbstractModel` y los datos se cargarán al modelo por medio de una diccionario. En la siguiente tabla aparecen los costes de transporte entre las ciudadaes y los almacenes.

In [2]:
d = {'Harlingen':{'NYC': 1956, 'LA': 1606, 'Chicago': 1410 , 'Houston': 330}, 
     'Memphis': {'NYC': 1096,'LA': 1792, 'Chicago': 531, 'Houston': 567}, 
     'Ashland': {'NYC': 485, 'LA': 2322, 'Chicago': 324, 'Houston': 1236 }
     }
pd.DataFrame.from_dict(d, orient='index')

,NYC,LA,Chicago,Houston
Harlingen,1956,1606,1410,330
Memphis,1096,1792,531,567
Ashland,485,2322,324,1236



Para terminar de completar este objeto tipo `dict`, crea un diccionario que relacione cada par ($w$,$c$) con el respectivo valor del coste.
```python
coste = {('Harlingen','NYC'): 1956,
         ('Harlingen','LA'): 1606,
         ...
         }
```

In [3]:
data = {None: {'W': ['Harlingen', 'Memphis', 'Ashland'], 
               'C': ['NYC', 'LA', 'Chicago', 'Houston'], 
               'P': {None: 2} ,
               'cost': {(w,c):d[w][c] for w in d for c in d[w]} 
               }}

## Implementación del modelo
El primer paso consiste en importar Pyomo y definir el tipo de modelo que se va a utilizar.

In [4]:
import pyomo.environ as pyo

model  = pyo.AbstractModel()

### Sets

In [5]:
model.C = pyo.Set(doc = 'Cities')
model.W = pyo.Set(doc = 'Warehouses')

### Variables

In [6]:
model.x = pyo.Var(model.W, model.C, bounds=(0,1))
model.y = pyo.Var(model.W, within=pyo.Binary)

### Parámetros

In [7]:
model.cost = pyo.Param(model.W, model.C, doc='Coste')
model.P = pyo.Param(doc='Número de almacenes')

### Funcion objetivo
$$
\begin{align}
\text{Coste total} & = \sum_{w\in W} \sum_{c\in C} \text{cost}_{wc} x_{wc} \nonumber
\end{align}
$$

In [8]:
def obj_rule(m):
    return sum(m.cost[w,c]*m.x[w,c] for w in m.W for c in m.C)
model.obj = pyo.Objective(rule=obj_rule)

### Restricciones
En este caso, se van a utilizar "reglas" para definir las restricciones. Cuando se utiliza el comando `Constraint(rule=myfunction)`, debe especificarse una función que toma como argumentos de entrada el modelo y los índices de los que depende la restricción.

- Cada tienda es abastecida exactamente por un almacén
$
\begin{align}
\sum_{w\in W} x_{wc} & = 1 \nonumber
\end{align}
$

In [9]:
def one_per_cust_rule(m, c):
    return sum(m.x[w,c] for w in m.W) == 1
model.one_per_cust = pyo.Constraint(model.C, rule=one_per_cust_rule)

- Número de almacenes que se desea construir
$
\begin{align}
\sum_{w\in W} y_{w} & = P \nonumber
\end{align}
$

In [10]:
def num_warehouses_rule(m):
    return sum(m.y[w] for w in m.W) <= m.P  
model.num_warehouses = pyo.Constraint(rule=num_warehouses_rule)

- Modelado con variables binarias
$$
\text{Coste} =
\begin{cases} 
x_{wc} = 0 & \text{si  } \; y_w=0 \\
0 \leq x_{wc} \leq 1 & \text{si  }\; y_w=1
\end{cases}
$$

In [11]:
def warehouse_active_rule(m, w, c):
    return m.x[w,c] <= m.y[w]
model.warehouse_active = pyo.Constraint(model.W, model.C, rule=warehouse_active_rule)

## Carga de datos al modelo 

In [12]:
instance = model.create_instance(data)

## Selección y llamada al solver
El algoritmo que se va a utilizar en la resolución del problema se llama con la orden `SolverFactory(solver_name)`, donde solver `solver_name` debe ser un dato de tipo cadena (string). En este caso, se va a utilizar el paquete GLPK (GNU Linear Programming Kit).

In [13]:
optimizer = pyo.SolverFactory('glpk')
results = optimizer.solve(instance)

## Inspeccionar la solución

In [14]:
print('Distribución óptima:')
instance.x.pprint()
print('Coste total:', instance.obj(),'  UM')

Distribución óptima:
x : Size=12, Index=W*C
    Key                      : Lower : Value : Upper : Fixed : Stale : Domain
      ('Ashland', 'Chicago') :     0 :   1.0 :     1 : False : False :  Reals
      ('Ashland', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
           ('Ashland', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
          ('Ashland', 'NYC') :     0 :   1.0 :     1 : False : False :  Reals
    ('Harlingen', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
    ('Harlingen', 'Houston') :     0 :   1.0 :     1 : False : False :  Reals
         ('Harlingen', 'LA') :     0 :   1.0 :     1 : False : False :  Reals
        ('Harlingen', 'NYC') :     0 :   0.0 :     1 : False : False :  Reals
      ('Memphis', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
      ('Memphis', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
           ('Memphis', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
          ('Memphis'

In [15]:
print(results.solver.status)
print(results.solver.termination_condition)

ok
optimal


## Análisis de sensibilidad

instance.P = 

In [16]:
optimizer = pyo.SolverFactory('glpk')
results = optimizer.solve(instance)

In [17]:
print('Distribución óptima')
instance.x.pprint()
print('Coste total:', instance.obj(),'  UM')

Distribución óptima
x : Size=12, Index=W*C
    Key                      : Lower : Value : Upper : Fixed : Stale : Domain
      ('Ashland', 'Chicago') :     0 :   1.0 :     1 : False : False :  Reals
      ('Ashland', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
           ('Ashland', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
          ('Ashland', 'NYC') :     0 :   1.0 :     1 : False : False :  Reals
    ('Harlingen', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
    ('Harlingen', 'Houston') :     0 :   1.0 :     1 : False : False :  Reals
         ('Harlingen', 'LA') :     0 :   1.0 :     1 : False : False :  Reals
        ('Harlingen', 'NYC') :     0 :   0.0 :     1 : False : False :  Reals
      ('Memphis', 'Chicago') :     0 :   0.0 :     1 : False : False :  Reals
      ('Memphis', 'Houston') :     0 :   0.0 :     1 : False : False :  Reals
           ('Memphis', 'LA') :     0 :   0.0 :     1 : False : False :  Reals
          ('Memphis',